### Ideas

* Standardize state
    * Use upper_bounds and lower_bounds where known ahead of time. (defined by environment, i.e. in sim)
    * Where bounds in a state dimension are unknown, scale by mean and std of everything in memory.
    * Consider scaling some dimensions if they are not normally distributed.
* Standardize and scale rewards
    * Try automatically detecting if scaling (e.g. logarathmic) results in a more normal distribution.
* Modulate noise while training
    * Try implementing this using hyperopt.
        * After each episode report training score and ask hyperopt for new noise parameters.
        
* After each episode:
    * Check if test score has improved
    * Save model weights if best score
    * If test_score > training_score, reduce epsilon
        

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings; warnings.simplefilter('ignore')

import math
import numpy as np
from ddpg_agent.contrib.physics_sim import PhysicsSim
from collections import namedtuple

class Task():
    """Task (environment) that defines the goal and provides feedback to the agent."""
    def __init__(self, init_pose=None, init_velocities=None, 
        init_angle_velocities=None, runtime=10., target_pos=None, 
        vert_dist_thresh=1, horiz_dist_thresh=1,
        target_steps_within_goal=1,):
        """Initialize a Task object.
        Params
        ======
            init_pose: initial position of the quadcopter in (x,y,z) dimensions and the Euler angles
            init_velocities: initial velocity of the quadcopter in (x,y,z) dimensions
            init_angle_velocities: initial radians/second for each of the three Euler angles
            runtime: time limit for each episode
            target_pos: target/goal (x,y,z) position for the agent
        """
        # Simulation
        self.sim = PhysicsSim(init_pose, init_velocities, init_angle_velocities, runtime) 
        # TODO: Make action_repeat align with agent.action_repeat
        self.action_repeat = 3

        self.state_size = self.action_repeat * 6 + 6
        self.observation_space = Space( list(list(self.sim.lower_bounds) + \
                                             [ -math.pi ]*3)*self.action_repeat + [float('-inf')]*6, 
                                       list(list(self.sim.upper_bounds) + \
                                            [ math.pi ]*3)*self.action_repeat + [float('inf')]*6 )
        self.action_space = Space([0,0,0,0], [900,900,900,900])
        self.action_size = 4

        # Goal
        self.target_pos = target_pos if target_pos is not None else np.array([0., 0., 10.])
        self.target_steps_within_goal = target_steps_within_goal
        self.steps_within_goal = 0
        self.horiz_dist_thresh = horiz_dist_thresh
        self.vert_dist_thresh = vert_dist_thresh
        
        # History
        self.step_history = []
        
    def reached_goal(self):
        horiz_distance_from_goal = np.sqrt((self.sim.pose[0]-self.target_pos[0])**2
                                           +(self.sim.pose[1]-self.target_pos[1])**2)
        vert_distance_from_goal = np.abs(self.sim.pose[2]-self.target_pos[2])
        return horiz_distance_from_goal < self.horiz_dist_thresh and \
                vert_distance_from_goal <= self.vert_dist_thresh
    
    def get_full_state(self):
        return namedtuple("QuadcopterState",[
                             'x', 'y', 'z', 'phi', 'theta', 'psi', 
                             'x_velocity', 'y_velocity', 'z_velocity', 
                             'phi_velocity', 'theta_velocity', 'psi_velocity',
                             'x_linear_accel','y_linear_accel','z_linear_accel',
                             'phi_angular_accel','theta_angular_accel','psi_angular_accel',
                            ])( *self.sim.pose, *self.sim.v, *self.sim.angular_v, 
                                *self.sim.linear_accel, *self.sim.angular_accels )

    def get_reward(self):
        """Uses current pose of sim to return reward."""
        #reward = 1.-.3*(abs(self.sim.pose[:3] - self.target_pos)).sum()
        reward = 0
        # Reward for staying at target altitude
#         target_alt=self.target_pos[2]
#         reward = .1*(target_alt - np.abs(self.sim.pose[2] - target_alt))/target_alt
#         distance_from_goal = np.sqrt((self.sim.pose[0]-self.target_pos[0])**2
#                  +(self.sim.pose[1]-self.target_pos[1])**2
#                  +(self.sim.pose[2]-self.target_pos[2])**2)
#         reward += 1-distance_from_goal/10.
        #Intermediate reward for flying at altitude
#         if np.abs(self.sim.pose[2] - self.target_pos[2]) < 1:
#             reward += 1
        vert_dist = np.abs(self.target_pos[2]-self.sim.pose[2])
        if vert_dist<10:
            reward += 10-vert_dist
            
        # Punish for high angular velocity
        reward -= (self.sim.angular_v[0]/30.)**2 + (self.sim.angular_v[1]/30.)**2
            
        # Punishment for crashing (altitude < 1 m)
#         if self.sim.pose[2]<=0: reward -= 1000
#         if self.sim.pose[2]<2: reward -= 1
        # Reward for being within goal radius
#         horiz_distance_from_goal = np.sqrt((self.sim.pose[0]-self.target_pos[0])**2
#                                            +(self.sim.pose[1]-self.target_pos[1])**2)
        # Reward for going up
        if self.sim.v[2]>0:
            reward += 1
        # Penalty for falling
        if self.sim.v[2]<0:
            reward -= .01*(self.sim.v[2]**2)
            
#         if self.reached_goal(): 
#             self.steps_within_goal += 1
#             reward += 1
# #             if self.steps_within_goal / self.action_repeat >= self.target_steps_within_goal: 
# #                 reward += 1000
#         else:
#             self.steps_within_goal = 0
        return reward

    def step(self, rotor_speeds):
        """Uses action to obtain next state, reward, done."""
        def zero_center_rotation(pose):
            pose[3]=pose[3]-math.pi
            pose[4]=pose[4]-math.pi
            pose[5]=pose[5]-math.pi
#             def _fix(p):
#                 if p>math.pi: p -= math.pi
#                 return p - math.pi/2.
#             for p in [3,4,5]:
#                 pose[p]=_fix(pose[p])
            
            return pose
        reward = 0
        pose_all = []
        for _ in range(self.action_repeat):
            done = self.sim.next_timestep(rotor_speeds) # update the sim pose and velocities
            reward += self.get_reward() 
#             pose_all.append(self.sim.pose)
            pose_all.append(zero_center_rotation(self.sim.pose))
#         reward = np.tanh(reward)
        next_state = list(np.concatenate(pose_all))+list(self.sim.v)+list(self.sim.angular_v)
#             import pdb; pdb.set_trace()
        # Punish and end episode for crashing
        if self.sim.pose[2]<=0: 
#             reward -= 100
            done = True
        
#         # Punish for excessive rotor speeds
#         reward -= np.sum(((rotor_speeds-self.action_space.low)/(self.action_space.high-self.action_space.low))**2)
        
        # end episode if at goal state
#         if self.steps_within_goal / self.action_repeat >= self.target_steps_within_goal: 
        if self.sim.pose[2] > self.target_pos[2]:
            reward += 1000
            done = True
        # Scale reward. 
        # TODO: How can the agent detect need for reward scaling automatically?
#         reward = np.log1p(reward) if reward>0 else np.log1p(-reward)

        return next_state, reward, done, None

    def reset(self):
        """Reset the sim to start a new episode."""
        self.sim.reset()
        state = list(np.concatenate([self.sim.pose] * self.action_repeat)) + \
                list(self.sim.v) + list(self.sim.angular_v)
        self.steps_within_goal = 0
        return state

class Space():
    def __init__(self, low, high):
        low = np.array(low)
        high = np.array(high)
        assert low.shape == high.shape,\
            "Expected bounds to be of same shape."
        self.low = low
        self.high = high
        self.shape = low.shape

In [ ]:
from ddpg_agent.agent import DDPG, Q_a_frames_spec
from ddpg_agent.visualizations import plot_quadcopter_episode

task = Task(init_pose=np.array([0., 0., 10, 0., 0., 0.]),
            #init_pose=np.array([0., 0., 10, math.pi/2., math.pi/2., 0.]), 
            init_velocities=np.array([0., 0., 0.]), 
            init_angle_velocities=np.array([0., 0., 0.]), 
            runtime=10., 
            vert_dist_thresh=1, horiz_dist_thresh=1, 
            target_steps_within_goal=25,
            target_pos=np.array([0., 0., 20.]),
           )

# q_a_frames_spec = Q_a_frames_spec(task, nx=16, ny=16, na=11, x_dim=3, y_dim=4, a_dim=0)

agent = DDPG(task, ou_mu=0, ou_theta=.02, ou_sigma=.5, 
             discount_factor=.7, replay_buffer_size=100000, replay_batch_size=512,
             tau_actor=.2, tau_critic=.4, 
#              relu_alpha_actor=.01, relu_alpha_critic=.01,
#              lr_actor=.00001, lr_critic=.0001, #activation_fn_actor='tanh',
#              l2_reg_actor=.01, l2_reg_critic=.01, 
#              bn_momentum_actor=0, bn_momentum_critic=.7, 
#              q_a_frames_spec=q_a_frames_spec, 
#              do_preprocessing=False,
#              input_bn_momentum_actor=.7,
#              input_bn_momentum_critic=.7,
#              activity_l2_reg=50,
#              output_action_regularizer=10,
            )
# agent.print_summary()

labels = ['time', 'x', 'y', 'z', 'phi', 'theta', 'psi', 'x_velocity',
          'y_velocity', 'z_velocity', 'phi_velocity', 'theta_velocity',
          'psi_velocity', 'rotor_speed1', 'rotor_speed2', 'rotor_speed3', 'rotor_speed4']
# [task.sim.time] + list(task.sim.pose) + list(task.sim.v) + list(task.sim.angular_v) + list(rotor_speeds)

In [ ]:
def episode_callback(episode_num):
    last_training_episode = agent.history.training_episodes[-1]
    last_test_episode = agent.history.test_episodes[-1]
    num_steps = last_training_episode.last_step - last_training_episode.first_step+1
    message = "Episode %i - epsilon: %7.4g, memory: %i, step: %i, training score: %6.2f, test score: %6.2f"\
                %(episode_num, 
                  last_training_episode.epsilon, 
                  len(agent.memory), 
                  num_steps, 
                  last_training_episode.score,
                  last_test_episode.score
                 )
    print(message)
    
    if episode_num%10==0:
        fig = plot_quadcopter_episode(last_training_episode)
        display(fig)
        
agent.set_episode_callback(episode_callback)

def max_training_score_callback(episode):
    last_training_episode = agent.history.training_episodes[-1]
    print("New best training score.")
    fig = plot_quadcopter_episode(last_training_episode)
    display(fig)
    
agent.set_max_training_score_callback(max_training_score_callback)
    
def max_test_score_callback(episode):
    last_test_episode = agent.history.test_episodes[-1]
    print("New best test score.")
    fig = plot_quadcopter_episode(last_test_episode)
    display(fig)
        
agent.set_max_test_score_callback(max_test_score_callback)

In [ ]:
agent.train_n_episodes(100, eps=200, eps_decay=1, run_tests=True )

In [ ]:
agent.train_n_episodes(100, eps=100, eps_decay=.5, run_tests=True )

In [ ]:
agent.train_n_episodes(100, eps=100, eps_decay=.25, run_tests=True )

In [ ]:
agent.train_n_episodes(100, eps=200, eps_decay=1, run_tests=True )

In [ ]:
agent.train_n_episodes(100, eps=400, eps_decay=0, run_tests=True )

In [ ]:
from ddpg_agent.visualizations import visualize_quad_agent

# q_a_frame = agent.get_q_a_frames(q_a_frames_spec)
agent.reset_episode()
visualize_quad_agent(agent, Q_a_frames_spec(task, nx=16, ny=16, na=11, x_dim=3, y_dim=4, a_dim=0))
visualize_quad_agent(agent, Q_a_frames_spec(task, nx=16, ny=16, na=11, x_dim=1, y_dim=2, a_dim=0))

In [ ]:
task.sim.init_pose=np.array([0,0,10,math.pi/2.,math.pi/2.,0])

In [ ]:
print(agent.memory.action_means)
print(np.sqrt(agent.memory.action_vars))

In [ ]:
agent.memory.state_means, np.sqrt(agent.memory.state_vars)

In [ ]:
agent.memory.reward_mean, np.sqrt(agent.memory.reward_var)

In [ ]:
np.max([s.reward for s in agent.memory.sample()])

In [ ]:
plt.hist([s.next_state[2] for s in agent.memory.sample()])

In [ ]:
np.std([s.action[1] for s in agent.memory.sample()])